# Boost and Rotate

In [1]:
import os
import sys
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

from ptetaphi_nn import PtEtaPhiNN
import tools
from keras.models import Sequential
from keras.layers import Dense, Dropout
with open("/home/cmccracken/start_tf/bbb/filepath.txt", 'r') as f:
    filename = f.read()

Using TensorFlow backend.


Load ntuple, get the data we need from the file

In [ ]:
s_table = tools.open_file(filename, sort_by="tag")

sorting data by tag


In [ ]:
print("filtering from", len(s_table), "total events")

# filter so we get events with 3 or 4 b-jets, and 3 tags
b34 = (s_table.nbjets == 3) | (s_table.nbjets == 4) # 3 or 4 b-jets
nt3 = s_table.nbtags==3  # 3 b-tags
events = s_table[b34 & nt3]

# and ensure that the 3 tags are actually correct
# this results in very little event loss
events = events[events.truth[:,0] == 1]
events = events[events.truth[:,1] == 1]
events = events[events.truth[:,2] == 1]

n_events = len(events)
print(n_events)

In [ ]:
cutoff = 10
# pad events out to 'cutoff' events
events = tools.pad(events, cutoff)
# boost/rotate
events = tools.boost_and_rotate(events)

In [ ]:
nn = PtEtaPhiNN(events)
nn.model = Sequential([
    Dense(3*(cutoff-3), input_dim=3*(cutoff-3), kernel_initializer='normal', activation='relu'),
    Dense(700, activation='relu'),
    Dropout(0.1),
    Dense(500, activation='relu'),
    Dropout(0.1),
    Dense(300, activation='relu'),
    Dropout(0.1),
    Dense(100, activation='relu'),
    Dropout(0.1),
    Dense( 50, activation='relu'),
    Dense(8, kernel_initializer='normal', activation='softmax')])
nn.model.compile(loss='categorical_crossentropy',
                 optimizer="adam", metrics=['acc'])
nn.model.summary()
nn.learn(epochs=1000)

In [ ]:
nn.evaluate(savename="boost_and_rotate_big")

In [ ]:
# evaluate on the set with >=4 jets, not ==4

# not sure if this is what we need, or the one above... Should ask
events2 = s_table[s_table.njets>=4 & nt3]
events2 = tools.pad(events2, length=cutoff)
events2 = tools.boost_and_rotate(events2)
nn.evaluate(events2)

In [ ]:
nn.model.save("bbb_model_boostrotate.json")

# get the architecture as a json string
arch = nn.model.to_json()
# save the architecture string to a file somehow, the below will work
with open('architecture.json', 'w') as arch_file:
    arch_file.write(arch)
# now save the weights as an HDF5 file
nn.model.save_weights('weights.h5')